In [1]:
import jieba
import pandas as pd
df_technology = pd.read_csv("./data/technology_news.csv", encoding='utf-8')
df_technology = df_technology.dropna()

df_car = pd.read_csv("./data/car_news.csv", encoding='utf-8')
df_car = df_car.dropna()

df_entertainment = pd.read_csv("./data/entertainment_news.csv", encoding='utf-8')
df_entertainment = df_entertainment.dropna()

df_military = pd.read_csv("./data/military_news.csv", encoding='utf-8')
df_military = df_military.dropna()

df_sports = pd.read_csv("./data/sports_news.csv", encoding='utf-8')
df_sports = df_sports.dropna()

technology = df_technology.content.values.tolist()[1000:21000]
car = df_car.content.values.tolist()[1000:21000]
entertainment = df_entertainment.content.values.tolist()[:20000]
military = df_military.content.values.tolist()[:20000]
sports = df_sports.content.values.tolist()[:20000]

In [2]:
stopwords=pd.read_csv("data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

## 去停用词：

In [3]:
def preprocess_text(content_lines, sentences, category):
    for line in content_lines:
        try:
            segs=jieba.lcut(line)
            segs = filter(lambda x:len(x)>1, segs)
            segs = filter(lambda x:x not in stopwords, segs)
            sentences.append((" ".join(segs), category))
        except Exception:
            print(line)
            continue 

#生成训练数据
sentences = []

preprocess_text(technology, sentences, 'technology')
preprocess_text(car, sentences, 'car')
preprocess_text(entertainment, sentences, 'entertainment')
preprocess_text(military, sentences, 'military')
preprocess_text(sports, sentences, 'sports')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\vivi\AppData\Local\Temp\jieba.cache
Loading model cost 1.321 seconds.
Prefix dict has been built successfully.


In [4]:
import random
random.shuffle(sentences)

In [5]:
for sentence in sentences[:10]:
    print(sentence[0], sentence[1])

中国 学生 体质 健康 水平 下降 警示 忧虑 多年 不绝于耳 现状 改变 学校 体育 突破口 sports
演练 部队 紧急 转移 炊事员 负重 过大 行动迟缓 暴露 目标 military
聚焦 国际化 议题 主会场 海航 生态 科技 消费者 业务 总裁 左力 万达 酒店 管理 公司 酒店 管理 总部 市场 销售部 总经理 于洋 天海 邮轮 执行 总裁兼 首席 战略 官魏裕 企业 发展 经历 讨论 中国 旅游 企业 全球 布局 国际化 进程 旅游业 大势所趋 深感 认同 世界 中餐 联合会 行业 专家 高福 中餐 美食 走向世界 怀抱 信心 期望 国际 娱乐 景区 运营 企业 海外 旅游局 代表 交流 国际 企业 中国 市场 策略 变迁 technology
摄影 指导 布拉德 鲁师 拍摄 作品 库克 美术 指导 克里斯蒂安 米雷兹 参与 喜剧 悬疑 歌舞 题材 凯撒 万岁 动作 奇幻 题材 古拉 元年 脍炙人口 电影 作品 entertainment
古佳耶 大提琴 北方 游牧民族 音乐 还于 国家大剧院 新闻 发布厅 隆重举行 CD 专辑 发布会 观众 现场 欣赏 到娜 木拉 老师 携手 艺术家 演奏 小宝贝 木卡姆 狂想曲 白色 毡房 专辑 作品 极富 创造力 音乐 精湛 演奏 深厚 北方 少数民族 文化 水准 唱片 制作 听众 带来 美好 游牧民族 文化 体验 entertainment
辽足 造血 功能强大 符合 U23 年龄段 球员 冯伯元 出场 sports
央视 解说 愤慨 sports
歌手 演出 该降 对此 有人 交给 市场 歌手 演出 商都 尊重 市场 涨价 赚钱 证明 涨得 道理 相关 人士 直言 歌手 降价 定价 不合理 虚荣 身价 价格 定得 太高 大部分 时间 待在家里 过气 歌手 二线 城市 没人 跑跑 三四 首歌 国外 唱够 小时 城市 肯定 市场 宗禾 entertainment
CBA 不到 半年 国奥 男篮 一员 第二年 夺得 亚青赛 MVP 球队 夺冠 CBA 职业生涯 第二年 年龄段 球员 征战 星锐 赛时 全明星 正赛 球员 广厦 内线 外援 霍尔 实力 慢慢 下降 林志杰 慢慢 老去 之时 球队 攻防 两端 核心 亮点 sports
中国互联网络信息中心 互联网 发展 研究部 主任 刘鑫 互联网 产业 影响 最先 

In [6]:
from sklearn.model_selection import train_test_split
x, y = zip(*sentences)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1234)

In [7]:
len(x_train)

65696

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(
    analyzer='word', # tokenise by character ngrams
    max_features=4000,  # keep the most common 1000 ngrams
)
vec.fit(x_train)

def get_features(x):
    vec.transform(x)

In [9]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)

MultinomialNB()

In [10]:
classifier.score(vec.transform(x_test), y_test)

0.8367505365541805

In [11]:
len(x_test)

21899

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(
    analyzer='word', # tokenise by character ngrams
    ngram_range=(1,4),  # use ngrams of size 1 and 2
    max_features=20000,  # keep the most common 1000 ngrams
)
vec.fit(x_train)

def get_features(x):
    vec.transform(x)

In [13]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)
classifier.score(vec.transform(x_test), y_test)

0.876067400337915

In [20]:
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


class TextClassifier():

    def __init__(self, classifier=MultinomialNB()):
        self.classifier = classifier
        self.vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,4), max_features=20000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

In [21]:
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict('这 是 有史以来 最 大 的 一 次 军舰 演习'))
print(text_classifier.score(x_test, y_test))

['military']
0.876067400337915


In [22]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')
svm.fit(vec.transform(x_train), y_train)
svm.score(vec.transform(x_test), y_test)

0.8475272843508836

In [ ]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(vec.transform(x_train), y_train)
svm.score(vec.transform(x_test), y_test)

In [ ]:
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


class TextClassifier():

    def __init__(self, classifier=SVC(kernel='linear')):
        self.classifier = classifier
        self.vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=12000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

In [ ]:
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict('这 是 有史以来 最 大 的 一 次 军舰 演习'))
print(text_classifier.score(x_test, y_test))